# Price Jump Detector Tool — Demo

Demonstrates the `PriceJumpDetectorTool` which computes four price-jump
metrics from local JSONL snapshot data:
1. **max_jump** — largest absolute consecutive price change
2. **mean_jump** — average absolute consecutive price change
3. **jump_count** — number of changes > |0.05|
4. **jump_density** — jump_count / n_steps

In [1]:
import sys, json
from pathlib import Path
from datetime import datetime, timezone

PROJECT_ROOT = Path.cwd().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

PREDICTION_AGENT_DIR = PROJECT_ROOT / "prediction_agent"
JSONL_PATH = PREDICTION_AGENT_DIR / "outputs" / "market_snapshots.jsonl"
print(f"JSONL: {JSONL_PATH}")

JSONL: /sessions/brave-inspiring-turing/mnt/prediction_agent/outputs/market_snapshots.jsonl


## 1. Load dataset and pick a market

In [2]:
rows = []
with open(JSONL_PATH) as f:
    for line in f:
        line = line.strip()
        if line:
            rows.append(json.loads(line))

from collections import Counter
counts = Counter(r["market_id"] for r in rows)
target = counts.most_common(1)[0][0]
print(f"Total rows: {len(rows)}")
print(f"Selected market: {target} ({counts[target]} rows)")

Total rows: 110
Selected market: STUB-NBA-LAL-BOS-001 (4 rows)


## 2. Run the tool

In [3]:
from prediction_agent.schemas import EventInput
from prediction_agent.tools.price_jump_detector_tool import PriceJumpDetectorTool

event = EventInput(event_id="demo", market_id=target, market_title="Demo", current_price=0.50)
tool = PriceJumpDetectorTool(jsonl_path=JSONL_PATH)
result = tool.run(event, window_minutes=999_999)

print("=== Tool Output ===")
print(f"tool_name    : {result.tool_name}")
print(f"output_vector: {result.output_vector}")
print(f"confidence   : {result.metadata['confidence']}")
print(f"sample_count : {result.metadata['sample_count']}")

labels = ["max_jump", "mean_jump", "jump_count", "jump_density"]
print("\nBreakdown:")
for label, val in zip(labels, result.output_vector):
    print(f"  {label:18s} = {val}")

=== Tool Output ===
tool_name    : price_jump_detector_tool
output_vector: [0.0, 0.0, 0.0, 0.0]
confidence   : 0.0
sample_count : 4

Breakdown:
  max_jump           = 0.0
  mean_jump          = 0.0
  jump_count         = 0.0
  jump_density       = 0.0


## 3. Plot price series with jumps highlighted

In [4]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

market_rows = [r for r in rows if r["market_id"] == target]

timestamps, prices = [], []
for r in market_rows:
    lp = r.get("last_price")
    bid, ask = r.get("yes_bid"), r.get("yes_ask")
    if lp is not None and lp > 0:
        price = lp
    elif bid is not None and ask is not None and (bid > 0 or ask > 0):
        price = (bid + ask) / 2.0
    else:
        continue
    timestamps.append(datetime.fromisoformat(r["timestamp"]))
    prices.append(price)

pairs = sorted(zip(timestamps, prices))
timestamps = [p[0] for p in pairs]
prices = [p[1] for p in pairs]

# Find jumps > 0.05
jump_ts, jump_pr = [], []
for i in range(len(prices) - 1):
    if abs(prices[i+1] - prices[i]) > 0.05:
        jump_ts.append(timestamps[i+1])
        jump_pr.append(prices[i+1])

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(timestamps, prices, marker="o", linewidth=1.5, markersize=4, color="#2563eb", label="price")
if jump_ts:
    ax.scatter(jump_ts, jump_pr, color="red", s=80, zorder=5, label=f"jumps > 0.05 ({len(jump_ts)})")
ax.set_title(f"Price with Jumps — {target}", fontsize=14)
ax.set_xlabel("Timestamp (UTC)")
ax.set_ylabel("Price")
ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
fig.autofmt_xdate()
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.savefig(str(PREDICTION_AGENT_DIR / "outputs" / "jumps_vs_time.png"), dpi=150)
plt.show()
print("Plot saved.")

Plot saved.
